### Code Hist.

 - CODE  
    &ensp; : KIER Data_동/층/호 List 생성  
    
- DESC  
    &ensp; : int_Domain 코드의 변경에 따라,   
    &emsp; KIER 에너지 사용량 데이터로부터 동/층/세대 리스트를 생성  

 - Hist  
    &ensp; 2024-01-25 Created  
    &ensp; 2024-06-11 코드개선 (코드 간략화)  
    &ensp; 2024-06-17 코드개선  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; - "동-층-호" HOUSE_ID 생성  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; - GAS Domain 추가

 

## 01. Code

### 01-01. Init

#### 01-01-01. Init_Module Import

#### Module import

In [11]:
#region Basic_Import
## Basic
import os, sys, warnings
os.path.dirname(os.path.abspath('__file__'))
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))
warnings.filterwarnings('ignore')

import numpy as np, pandas as pd
from pandas import DataFrame, Series
import math, random

## Datetime
import time
import datetime as dt
from datetime import datetime, date, timedelta

import glob
from glob import glob
import requests
import json

## 시각화
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]

from scipy import stats

# K-Means 알고리즘
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.model_selection import train_test_split

# CLustering 알고리즘의 성능 평가 측도
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score, adjusted_rand_score, silhouette_score, rand_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.metrics.cluster import contingency_matrix

## 정규화
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn import metrics

import urllib
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

## Init.
pd.options.display.float_format = '{:.10f}'.format
#endregion Basic_Import

In [12]:
## Import_DL
str_tar = "tf"
## For Torch
if str_tar == "torch":
    import torch
    import torch.nn as nn
    from torch.nn.utils import weight_norm
    print("Torch Imported")
## For TF
elif str_tar == "tf":
    import tensorflow as tf
    import tensorflow_addons as tfa
    from keras.callbacks import EarlyStopping, ModelCheckpoint
    from keras.models import Sequential, load_model
    from keras_flops import get_flops
    print("Tensorflow Imported")
else:
    print("Error : Cannot be used except for Keywords")
    print(" : torch / tf")

Tensorflow Imported


In [1]:
## Import_Local
# from Src_Dev_Common import DEV_Common_Data_Datetime as com_date
# from Src_Dev_Common import DEV_KASI_Holiday as com_Holi
# from Src_Dev_Common import DEV_Common_Data_Analysis as com_Analysis
# from Src_Dev_Common import DEV_Common_Data_Preprocessing as com_Prep
# from Src_Dev_Common import DEV_Common_Model as com_Model
# from Src_Dev_Common import DEV_Common_Data_Visualization as com_Visual
# from Src_Dev_Common import DEV_KMA_Weather_ASOS as com_ASOS
# from Src_Dev_Common import DEV_KDHC_Usage as com_KDHC
# from Src_Dev_Common import DEV_KIER_Usage as com_KIER

### 01-01-02. Config (Directory, Params)

In [14]:
## Init_config
SEED = 42

np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = "1"
random.seed(SEED)

In [15]:
## Define Todate str
str_now_ymd = pd.datetime.now().date()
str_now_y, str_now_m, str_now_d = pd.datetime.now().year, pd.datetime.now().month, pd.datetime.now().day
str_now_hr, str_now_min = pd.datetime.now().hour, pd.datetime.now().minute

print(pd.datetime.now())
print(str(str_now_y) + " / " + str(str_now_m)  + " / " + str(str_now_d))
print(str(str_now_hr) + " : " + str(str_now_min))

2024-06-27 15:01:21.692008
2024 / 6 / 27
15 : 1


In [16]:
## Dict_Domain
dict_domain = {0:"ELEC", 1:"HEAT", 2:"WATER", 3:"HOT_HEAT", 4:"HOT_FLOW", 5:"GAS"} ## GAS는 사용하지 않음.
int_domain = 5
str_domain = str(dict_domain[int_domain])

dict_col_accu = {0 : "ACTUAL_ACCU_EFF" ## ELEC
                 , 1 : "ACCU_HEAT" ## HEAT
                 , 2 : "ACCU_FLOW" ## WATER
                 , 3 : "ACCU_HEAT" ## HOT 열량
                 , 4 : "ACCU_FLOW" ## HOT 유량
                 , 5 : "ACCU_FLOW" ## GAS
                 }
str_col_accu = str(str_domain + "_" + str(dict_col_accu[int_domain]))

dict_col_inst = {0 : "INST_EFF" ## ELEC_INST_EFF
                , 1 : "INST_HEAT" ## HEAT_INST_HEAT
                , 2 : "INST_FLOW" ## WATER_INST_FLOW
                , 3 : "INST" ## HOT_HEAT_INST
                , 4 : "INST" ## HOT_FLOW_INST
                , 5 : "INST_FLOW" ## GAS_INST_FLOW
                } 
str_col_inst = str(str_domain + "_" + str(dict_col_inst[int_domain]))

## Directory Root
str_dirData = "../data/data_Energy_KIER/"
str_dir_raw = '../data/data_Energy_KIER/KIER_0_Raw/'
str_dirName_bld = '../data/data_Energy_KIER/KIER_1_BLD/'
str_dirName_h = '../data/data_Energy_KIER/KIER_2_H_' + str_domain + '/'

## File
str_fileRaw = str('KIER_RAW_' + str_domain + '_2024-06-07.csv')
if int_domain == 5: str_fileRaw_hList = str('KIER_hList_Common.csv') 
else : str_fileRaw_hList = str('KIER_hList_' + str_domain + '.csv')

print(str(os.listdir(str_dirData)) + "\n")
print(os.listdir(str_dirName_h))

['.ipynb_checkpoints', 'BS_CONFIGURATION_202309251452.csv', 'DATE_1M_2023-10-20.csv', 'KIER 전처리 현황_2024-06-25.xlsx', 'KIER_0_Raw', 'KIER_1_BLD', 'KIER_2_H_ELEC', 'KIER_2_H_GAS', 'KIER_2_H_HEAT', 'KIER_2_H_HOT_FLOW', 'KIER_2_H_HOT_HEAT', 'KIER_2_H_WATER', 'KIER_ASOS_WEATHER_DAILY_202309251521.csv', 'KIER_ASOS_WEATHER_HOUR_202309251521.csv', 'KIER_DATA_OLD', 'KIER_ETC', 'KIER_hList_Comparison_2024-06-26.xlsx', 'KIER_List_Table_Column_2023-09-25.xlsx', 'KIER_Query_2023-09-25.txt', 'KMA_ASOS_119_2010_2023_1st_to CSV.csv', '[IITP] 데이터 테이블 정리 (공유 원본).docx', '~$KIER 전처리 현황_2024-06-25.xlsx']

['KIER_GAS_561-1-1_ACCU_01_Raw.csv', 'KIER_GAS_561-1-1_INST_01_10min.csv', 'KIER_GAS_561-1-2_ACCU_01_Raw.csv', 'KIER_GAS_561-1-2_INST_01_10min.csv', 'KIER_GAS_561-1-3_ACCU_01_Raw.csv', 'KIER_GAS_561-1-3_INST_01_10min.csv', 'KIER_GAS_561-1-4_ACCU_01_Raw.csv', 'KIER_GAS_561-1-4_INST_01_10min.csv', 'KIER_GAS_561-10-1_ACCU_01_Raw.csv', 'KIER_GAS_561-10-1_INST_01_10min.csv', 'KIER_GAS_561-10-2_ACCU_01_Raw.csv'

In [17]:
## HOT_HEAT와 HOT_FLOW가 한 시트에 존재하는 경우
if int_domain == 3 or int_domain == 4 :
    str_fileRaw = 'KIER_RAW_HOT_2024-06-07.csv'
    df_hot_tmp = pd.read_csv(str_dir_raw + str_fileRaw, index_col = 0)
    df_hot_tmp = df_hot_tmp.reset_index().rename({'HOT_ACCU_HEAT':'HOT_HEAT_ACCU', 'HOT_ACCU_FLOW':'HOT_FLOW_ACCU'}, axis=1)
    ## 온수 열량
    df_hot_tmp[['METER_DATE', 'HOUSE_ID_DONG', 'HOUSE_ID_HO_PRE', 'HOUSE_ID_HO', 'HOT_HEAT_ACCU']].to_csv(str_dir_raw + 'KIER_RAW_HOT_HEAT_2024-06-07.csv')
    ## 온수 유량
    df_hot_tmp[['METER_DATE', 'HOUSE_ID_DONG', 'HOUSE_ID_HO_PRE', 'HOUSE_ID_HO', 'HOT_FLOW_ACCU']].to_csv(str_dir_raw + 'KIER_RAW_HOT_FLOW_2024-06-07.csv')
    print(df_hot_tmp.head(5))

In [18]:
str_dir_raw + str_fileRaw

'../data/data_Energy_KIER/KIER_0_Raw/KIER_RAW_GAS_2024-06-07.csv'

In [19]:
## Gas의 경우는 ParserError: Error tokenizing data 발생하므로 아래와 같이 Load
df_kier_hList = pd.read_csv(str_dir_raw + str_fileRaw, index_col= 0, error_bad_lines = False).reset_index()
## 단, warn_bad_lines는 True로 그대로 두어 어느 행이 사라졌는지 확인해야만 함

print(df_kier_hList.shape, " /// ", df_kier_hList.columns)
df_kier_hList

Skipping line 52380994: expected 5 fields, saw 6



(52380992, 5)
Index(['METER_DATE', 'HOUSE_ID_DONG', 'HOUSE_ID_HO_PRE', 'HOUSE_ID_HO',
       'GAS_ACCU_FLOW'],
      dtype='object')
            METER_DATE  HOUSE_ID_DONG  HOUSE_ID_HO_PRE  \
0  2000-10-00 00:00:00              0                0   
1  2000-11-00 00:00:00              0                0   
2  2000-12-00 00:00:00              0                0   
3  2000-13-00 00:00:00              0                0   
4  2000-16-00 00:00:00              0                0   

                                    HOUSE_ID_HO  GAS_ACCU_FLOW  
0  NkU0NTM2MTgwMzc4MDI1NEYyMjhFMEM4NEJDREVCOEY=   3.3800000000  
1  NkU0NTM2MTgwMzc4MDI1NEYyMjhFMEM4NEJDREVCOEY=   3.3800000000  
2  NkU0NTM2MTgwMzc4MDI1NEYyMjhFMEM4NEJDREVCOEY=   3.3800000000  
3  NkU0NTM2MTgwMzc4MDI1NEYyMjhFMEM4NEJDREVCOEY=   3.3800000000  
4  NkU0NTM2MTgwMzc4MDI1NEYyMjhFMEM4NEJDREVCOEY=   3.3800000000  


In [20]:
## list : Bld, Floor, House
df_kier_hList = df_kier_hList[['HOUSE_ID_DONG', 'HOUSE_ID_HO_PRE', 'HOUSE_ID_HO']].drop_duplicates()
df_kier_hList['HOUSE_ID_HO_INT'] = 0

In [21]:
## list : Bld
list_Bld = df_kier_hList[['HOUSE_ID_DONG']].drop_duplicates()['HOUSE_ID_DONG']
for bld in list_Bld:
    ## list : F
    list_F = df_kier_hList[df_kier_hList['HOUSE_ID_DONG'] == bld]['HOUSE_ID_HO_PRE'].drop_duplicates()
    
    for f in list_F:
        list_H = df_kier_hList[(df_kier_hList['HOUSE_ID_DONG'] == bld)
                                & (df_kier_hList['HOUSE_ID_HO_PRE'] == f)]['HOUSE_ID_HO'].drop_duplicates()

        ## int_h는 해시코드 형태로 나타나므로, 
        ## 숫자를 부여하여 호실로서 재정의 후 df_kier_hList 내에 저장
        int_house_num = 1
        for h in list_H:
            idx_h = df_kier_hList[(df_kier_hList['HOUSE_ID_DONG'] == bld)
                                    & (df_kier_hList['HOUSE_ID_HO_PRE'] == f)
                                    & (df_kier_hList['HOUSE_ID_HO'] == h)].index
            df_kier_hList['HOUSE_ID_HO_INT'].loc[idx_h] = int_house_num

            int_house_num = int_house_num + 1
df_kier_hList = df_kier_hList.reset_index()
# df_kier_hList = df_kier_hList[['HOUSE_ID_DONG', 'HOUSE_ID_HO_PRE', 'HOUSE_ID_HO_INT', 'HOUSE_ID_HO']]

col_house_id = ['HOUSE_ID_DONG', 'HOUSE_ID_HO_PRE', 'HOUSE_ID_HO_INT']
df_kier_hList['HOUSE_ID'] = df_kier_hList[col_house_id].apply(lambda row : '-'.join(row.values.astype(str)), axis = 1)
df_kier_hList = df_kier_hList[['HOUSE_ID_DONG', 'HOUSE_ID_HO_PRE', 'HOUSE_ID_HO_INT', 'HOUSE_ID', 'HOUSE_ID_HO']]

df_kier_hList.to_csv(str_dir_raw + str_fileRaw_hList)
df_kier_hList

,HOUSE_ID_DONG,HOUSE_ID_HO_PRE,HOUSE_ID_HO_INT,HOUSE_ID,HOUSE_ID_HO
0,0,0,1,0-0-1,NkU0NTM2MTgwMzc4MDI1NEYyMjhFMEM4NEJDREVCOEY=
1,0,10,1,0-10-1,OUU3RkE4MEExRTc2NUVEMUM1NTAxQTQxMTZGNzEyMDc=
2,0,11,1,0-11-1,MjQ0NjAwMkZFMEMzNkE5OUEyQzg1NzhCNURCRjM5OTc=
3,0,12,1,0-12-1,MkFFRTgwMjk1NDA4NDAwODk5NTJDRUVCOTIxM0Y4NkQ=
4,0,16,1,0-16-1,M0ZFOTgyMUQ4N0U1QjAzNDJFNUNGOTE0NkNBRDgxOEM=
...,...,...,...,...,...
450,1433,5,1,1433-5-1,MDRDQkM5NkE1MjVDQ0I0M0VDRkU0QUFERjYyQzY0MTE=
451,1436,3,1,1436-3-1,NDhDRENGM0JGMzZDRUU2MUE2NjM0RTE1NDk0MjFFRTA=
452,1446,52,1,1446-52-1,M0EzRDNGMDBDQTk3N0I0MDNDNzM3QUM5Nzg0M0ZCQkM=
453,1446,54,1,1446-54-1,Njg0MTdGMDNDOEMxMzM2QzhGODBGNkMxRjgyNUY2NzQ=
